# Basic Text Regressor using Auto-Keras

Basic implementatio for toxic comment challange.

In [ ]:
import sys
package_dir = '../input/autokeras-april-2021'
sys.path.insert(0, package_dir)

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak

Load data

In [ ]:
df = pd.read_csv("../input/jigsaw-score-augumentation/jigsaw_train_hate_annotationprob.csv",index_col=0)


In [ ]:
df.describe()

In [ ]:
ruddit_df = pd.read_csv("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv",index_col=0)


In [ ]:
ruddit_df.head()

In [ ]:
ruddit_df.describe()

In [ ]:
df['target']=df['proposed_score3']+0.1*df['offensive_agreement_rating']

In [ ]:
df= df[['text', 'target']]

In [ ]:
#df["target"]=df['target']+0.066618

In [ ]:
# ruddit_df['offensiveness_score']=ruddit_df['offensiveness_score']+0.889000

# ruddit_df['offensiveness_score']=ruddit_df['offensiveness_score']*3.1091

In [ ]:
ruddit_df.rename(columns={'txt':'text','offensiveness_score':'target'},inplace=True)


In [ ]:

ruddit_df= ruddit_df[['text', 'target']]

Train test split. Not used/usefull. 

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.10)
#valid, test = train_test_split(valid, test_size=0.3)

In [ ]:
df_test=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)

In [ ]:
X_less_toxic =df_test.less_toxic.values
X_more_toxic =df_test.more_toxic.values
X_comments_to_score =comments_to_score.text.values

In [ ]:
train_data = train['text'].values
train_target = train['target'].values

valid_data = valid['text'].values
valid_target = valid['target'].values

print(train_data.shape, train_target.shape)
print(valid_data.shape, valid_target.shape)

Simple model 0.6745383286834064 

* validation score with second conv block and 3 epoch:
0.6651720472963997


Prepare a keras model. 

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
#output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg.fit(train_data, train_target, epochs=5,validation_split=0.15)

Validate results

In [ ]:
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p1 = reg.predict(X_less_toxic).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p2 = reg.predict(X_more_toxic).squeeze().tolist()


In [ ]:
def compare(p1,p2):
    z=0
    k=0
    n=len(p1)
    for i in range(len(p1)):
        if p1[i]< p2[i]:
            z=z+1
        else:
            k=k+1
            #print('nu')
    print(z/n)
    return z/n

In [ ]:
# Validation Accuracy
compare(p1,p2)

In [ ]:
df_test['score_max']=p2
df_test['score_min']=p1
df_test['diff']=df_test['score_max']-df_test['score_min']



In [ ]:
def correctie(diff):
    if diff<0:
        return diff+0.01
    else:
        return 0

In [ ]:
df_test['err_corr']=df_test['diff'].apply(lambda x: correctie(x))

In [ ]:
df_test['err_corr'].describe()

In [ ]:
train_data1 = df_test['more_toxic'].values
train_target1 = df_test['err_corr'].values


In [ ]:
# Initialize the text regressor.
#reg1 = ak.TextRegressor(overwrite=True, max_trials=10)  # It tries 10 different models.
reg1 = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=2)
reg1.fit(train_data1, train_target1, epochs=5,validation_split=0.15)

In [ ]:
p3[0]

In [ ]:
p3 = reg1.predict(X_less_toxic).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p4 = reg1.predict(X_more_toxic).squeeze().tolist()

In [ ]:
compare(p3,p4)

In [ ]:
compare(np.asarray(p1)-np.asarray(p3),np.asarray(p2)-np.asarray(p4))

In [ ]:
compare(np.asarray(p1)+np.asarray(p3),np.asarray(p2)+np.asarray(p4))

In [ ]:
compare(np.asarray(p1)+np.asarray(p3),np.asarray(p2))

In [ ]:
compare(p1+p3,p2+p4)

In [ ]:
df_test['score_max']=np.asarray(p2)+np.asarray(p4)
df_test['score_min']=np.asarray(p1)+np.asarray(p3)
df_test['diff']=df_test['score_max']-df_test['score_min']

In [ ]:
df_test['err_corr']=df_test['diff'].apply(lambda x: correctie(x))

In [ ]:
df_test['err_corr'].describe()

In [ ]:
train_data2 = df_test['less_toxic'].values
train_target2 = -1*df_test['err_corr'].values


In [ ]:
reg2 = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg2.fit(train_data2, train_target2, epochs=5,validation_split=0.15)

In [ ]:
p5 = reg2.predict(X_less_toxic).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p6 = reg2.predict(X_more_toxic).squeeze().tolist()

In [ ]:
compare(p5,p6)

In [ ]:
compare(np.asarray(p1)+np.asarray(p5),np.asarray(p2)+np.asarray(p6))

In [ ]:
compare(np.asarray(p1)+np.asarray(p3)+np.asarray(p5),np.asarray(p2)+np.asarray(p4)+np.asarray(p6))

In [ ]:
compare(np.asarray(p1)+np.asarray(p3),np.asarray(p2)+np.asarray(p6))

In [ ]:
df_test.head()

In [ ]:
df_test['score_max']=np.asarray(p1)+np.asarray(p3)
df_test['score_min']=np.asarray(p2)+np.asarray(p6)
df_test['diff']=df_test['score_max']-df_test['score_min']

In [ ]:
df_test['err_corr']=df_test['diff'].apply(lambda x: correctie(x))

In [ ]:
train_data3 = df_test['less_toxic'].values
train_target3 = -1*df_test['err_corr'].values

In [ ]:
df_test['err_corr'].describe()

In [ ]:
reg2 = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg2.fit(train_data3, train_target3, epochs=5,validation_split=0.15)

In [ ]:
p7 = reg2.predict(X_less_toxic).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p8 = reg2.predict(X_more_toxic).squeeze().tolist()

In [ ]:
compare(p7,p8)

In [ ]:
compare(np.asarray(p1)+np.asarray(p3),np.asarray(p2)+np.asarray(p6)+np.asarray(p8))

In [ ]:
df_test['score_max']=np.asarray(p2)+np.asarray(p6)+np.asarray(p8)
df_test['score_min']=np.asarray(p1)+np.asarray(p3)
df_test['diff']=df_test['score_max']-df_test['score_min']

In [ ]:
df_test['err_corr']=df_test['diff'].apply(lambda x: correctie(x))

In [ ]:
df_test['err_corr'].describe()

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('validation_data_pseudo_scores.csv')

In [ ]:
clasament_df=df_test[['less_toxic', 'score_min']]
clasament_df.rename(columns={'less_toxic':'text','score_min':'score'},inplace=True)
clasament_df1=df_test[['more_toxic', 'score_max']]
clasament_df1.rename(columns={'more_toxic':'text','score_max':'score'},inplace=True)

In [ ]:
clasament=pd.concat([clasament_df, clasament_df1])

In [ ]:
len(clasament)

In [ ]:
clasament.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np

sns.set(rc={"figure.figsize": (30, 4)})
x = clasament['score']
ax = sns.distplot(x)
plt.show()

In [ ]:
clasament.to_csv('validation_data_pseudo_scores_clasament.csv')

In [ ]:
# Get the predictions 
scored_sub=reg.predict(X_comments_to_score).squeeze().tolist()

In [ ]:
comments_to_score['score']=scored_sub

In [ ]:
submission=comments_to_score[['score']]

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.head()

Ruddit -dumb test

In [ ]:
# from sklearn.model_selection import train_test_split

# train, valid = train_test_split(ruddit_df, test_size=0.10)
# #valid, test = train_test_split(valid, test_size=0.3)

In [ ]:
# train_data = train['text'].values
# train_target = train['target'].values

# valid_data = valid['text'].values
# valid_target = valid['target'].values

# print(train_data.shape, train_target.shape)
# print(valid_data.shape, valid_target.shape)

In [ ]:
# input_node = ak.TextInput()
# output_node = ak.TextToIntSequence()(input_node)
# output_node = ak.Embedding()(output_node)
# # Use separable Conv layers in Keras.
# output_node = ak.ConvBlock(separable=True)(output_node)
# #output_node = ak.ConvBlock(separable=True)(output_node)
# output_node = ak.RegressionHead()(output_node)
# reg = ak.AutoModel(
#     inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
# )
# reg.fit(train_data, train_target, epochs=10,validation_split=0.12)

In [ ]:
# input_node = ak.TextInput()
# output_node2 = ak.TextBlock(block_type="transformer",pretraining="fasttext")(input_node)
# output_node3 = ak.TextBlock(block_type="ngram")(input_node)
# output_node1 = ak.TextToIntSequence()(input_node)
# output_node2 = ak.Embedding()(output_node2)
# output_node1 = ak.Embedding()(output_node1)
# #output_node = ak.ConvBlock(separable=True)(output_node)
# output_node = ak.Merge()([output_node1, output_node2,output_node3 ])
# output_node = ak.RegressionHead()(output_node)
# reg = ak.AutoModel(
#     inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
# )
# reg.fit(train_data, train_target, epochs=6)

In [ ]:
# #preds = reg.predict(X_less_toxic).squeeze().tolist()
# #p1 = predictor.predict(X_less_toxic)
# p3 = reg.predict(X_less_toxic).squeeze().tolist()
# #p2 = predictor.predict(X_more_toxic)
# p4 = reg.predict(X_more_toxic).squeeze().tolist()

In [ ]:
# # Validation Accuracy
# compare(p3,p4)

In [ ]:
# compare(p4,p3)

In [ ]:
# compare(p1+p3,p2+p4)

In [ ]:
# # Get the predictions 
# scored_sub1=reg.predict(X_comments_to_score).squeeze().tolist()

In [ ]:
# comments_to_score['score']=scored_sub1+scored_sub

In [ ]:
# submission=comments_to_score[['score']]

In [ ]:
# submission.to_csv('submission.csv')